# Automated Guitar Amp Modelling Training on Google Colab

This notebook sets up and runs the neural network training for guitar amplifier/distortion pedal modelling using PyTorch.

## Setup Instructions:
1. Make sure you have GPU runtime enabled: Runtime → Change runtime type → Hardware accelerator → GPU
2. Upload your training data files to Colab (or use the provided generate_test_signals.py)
3. Run the cells in order

In [ ]:
# Check GPU availability
import torch
print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    print(f"CUDA version: {torch.version.cuda}")
else:
    print("WARNING: No GPU detected. This training will be very slow on CPU!")

In [ ]:
# Install required dependencies
!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu121
!pip install numpy scipy matplotlib pyyaml tqdm librosa tensorboard
!pip install pytest black mypy  # Optional development tools

In [ ]:
# Clone the repository (remove old clone if it exists)
import os
import shutil

# Clean up any old clones
if os.path.exists('Automated-GuitarAmpModelling-3'):
    print("Removing old clone...")
    shutil.rmtree('Automated-GuitarAmpModelling-3', ignore_errors=True)

if os.path.exists('MonsterLSTM_Capture'):
    print("MonsterLSTM_Capture already exists, skipping clone...")
else:
    # Clone the repository
    !git clone https://github.com/diogoguedes666/MonsterLSTM_Capture.git

# Navigate to the repository directory
%cd MonsterLSTM_Capture

# Verify we're in the right directory and CoreAudioML exists
print(f"Current directory: {os.getcwd()}")
print(f"CoreAudioML exists: {os.path.exists('CoreAudioML')}")
if os.path.exists('CoreAudioML'):
    print(f"CoreAudioML contents: {os.listdir('CoreAudioML')[:5]}")
else:
    print("⚠️ CoreAudioML not found! Listing current directory:")
    print(os.listdir('.')[:10])

# Alternative: If you want to upload your local files instead of cloning
# from google.colab import files
# uploaded = files.upload()  # Upload your project files

In [ ]:
# Create necessary directories
!mkdir -p Data/train Data/val Data/test Results
!mkdir -p Configs

In [ ]:
# Option 1: Generate test signals if you don't have real training data
# This creates synthetic audio data for testing the training pipeline

import numpy as np
import scipy.io.wavfile as wavfile
import os

def generate_test_audio(filename, duration=10, sample_rate=44100, freq=440):
    """Generate a simple test audio file"""
    t = np.linspace(0, duration, int(sample_rate * duration), False)
    # Create a mix of sine waves for more interesting content
    audio = 0.3 * np.sin(freq * 2 * np.pi * t)  # 440 Hz sine wave
    audio += 0.2 * np.sin(2 * freq * 2 * np.pi * t)  # 880 Hz harmonic
    audio += 0.1 * np.sin(3 * freq * 2 * np.pi * t)  # 1320 Hz harmonic

    # Normalize to prevent clipping
    audio = audio / np.max(np.abs(audio)) * 0.8

    # Convert to 16-bit PCM
    audio_int16 = (audio * 32767).astype(np.int16)

    wavfile.write(filename, sample_rate, audio_int16)
    print(f"Generated test audio: {filename}")

# Generate test data
generate_test_audio("Data/train/dls2-input.wav", duration=30)
generate_test_audio("Data/train/dls2-target.wav", duration=30)  # This would be your "amp output"

generate_test_audio("Data/val/dls2-input.wav", duration=10)
generate_test_audio("Data/val/dls2-target.wav", duration=10)

generate_test_audio("Data/test/dls2-input.wav", duration=5)
generate_test_audio("Data/test/dls2-target.wav", duration=5)

print("Test data generation complete!")
print("Note: For real amp modelling, replace these with actual guitar -> amp recordings")

In [ ]:
# Option 2: Upload your own training data
# Easy upload: Upload a zip file containing your Data folder structure
# OR upload individual files one by one

from google.colab import files
import os
import zipfile
import shutil

# Ensure Data directories exist
os.makedirs('Data/train', exist_ok=True)
os.makedirs('Data/val', exist_ok=True)
os.makedirs('Data/test', exist_ok=True)

print("📤 Upload your training data")
print("\nOption A: Upload a ZIP file containing your Data folder")
print("   Your ZIP should contain: Data/train/, Data/val/, Data/test/")
print("   Or: Data/train/dls2-input.wav, Data/train/dls2-target.wav, etc.")
print("\nOption B: Upload individual files (will be prompted for each)")
print("\nChoose:")
print("1. Upload ZIP file (recommended)")
print("2. Upload individual files")

choice = input("Enter choice (1 or 2): ").strip()

if choice == "1":
    # Upload and extract ZIP file
    print("\n📦 Upload your ZIP file containing the Data folder...")
    uploaded = files.upload()
    
    zip_filename = None
    for filename in uploaded.keys():
        if filename.endswith('.zip'):
            zip_filename = filename
            break
    
    if zip_filename:
        print(f"\n📂 Extracting {zip_filename}...")
        with zipfile.ZipFile(zip_filename, 'r') as zip_ref:
            zip_ref.extractall('.')
        
        # Check if Data folder was extracted
        if os.path.exists('Data'):
            print("✅ Data folder found!")
            
            # Verify structure
            required_files = [
                'Data/train/dls2-input.wav',
                'Data/train/dls2-target.wav',
                'Data/val/dls2-input.wav',
                'Data/val/dls2-target.wav',
                'Data/test/dls2-input.wav',
                'Data/test/dls2-target.wav'
            ]
            
            missing = []
            for filepath in required_files:
                if not os.path.exists(filepath):
                    missing.append(filepath)
            
            if missing:
                print(f"\n⚠️  Missing files:")
                for f in missing:
                    print(f"   - {f}")
                print("\nMake sure your ZIP contains these files.")
            else:
                print("\n✅ All required files found!")
                print("\n📁 Data structure:")
                for split in ['train', 'val', 'test']:
                    files_in_split = os.listdir(f'Data/{split}')
                    print(f"   Data/{split}/: {files_in_split}")
        else:
            print("⚠️  Data folder not found in ZIP. Checking extracted contents...")
            print(f"   Extracted files: {os.listdir('.')[:10]}")
        
        # Clean up zip file
        os.remove(zip_filename)
    else:
        print("❌ No ZIP file found in upload!")

else:
    # Individual file upload (original method)
    print("\n📤 Upload your training input WAV file (guitar signal)")
    uploaded = files.upload()
    for filename in uploaded.keys():
        os.rename(filename, f"Data/train/dls2-input.wav")
        print(f"✓ Saved as: Data/train/dls2-input.wav")

    print("\n📤 Upload your training target WAV file (amp output)")
    uploaded = files.upload()
    for filename in uploaded.keys():
        os.rename(filename, f"Data/train/dls2-target.wav")
        print(f"✓ Saved as: Data/train/dls2-target.wav")

    print("\n📤 Upload validation input WAV file")
    uploaded = files.upload()
    for filename in uploaded.keys():
        os.rename(filename, f"Data/val/dls2-input.wav")
        print(f"✓ Saved as: Data/val/dls2-input.wav")

    print("\n📤 Upload validation target WAV file")
    uploaded = files.upload()
    for filename in uploaded.keys():
        os.rename(filename, f"Data/val/dls2-target.wav")
        print(f"✓ Saved as: Data/val/dls2-target.wav")

    print("\n📤 Upload test input WAV file")
    uploaded = files.upload()
    for filename in uploaded.keys():
        os.rename(filename, f"Data/test/dls2-input.wav")
        print(f"✓ Saved as: Data/test/dls2-input.wav")

    print("\n📤 Upload test target WAV file")
    uploaded = files.upload()
    for filename in uploaded.keys():
        os.rename(filename, f"Data/test/dls2-target.wav")
        print(f"✓ Saved as: Data/test/dls2-target.wav")

print("\n✅ Upload complete!")
print("\n📝 Note: Make sure your WAV files are:")
print("   - Mono (1 channel)")
print("   - 44.1 kHz sample rate")
print("   - Input and target files are the same length")
print("   - Input: clean guitar signal")
print("   - Target: amp/effect output")

In [ ]:
# Create RNN3.json config file
config_content = '''{
  "model": "SimpleRNN",
  "input_size": 1,
  "output_size": 1,
  "num_blocks": 2,
  "hidden_size": 32,
  "unit_type": "LSTM",
  "skip_con": true,
  "segment_length": 22050,
  "batch_size": 512,
  "epochs": 100,
  "learn_rate": 0.005,
  "validation_f": 5,
  "validation_p": 20,
  "loss_fcns": {"ESRPre": 0.9, "DC": 0.1},
  "pre_filt": "high_pass",
  "cuda": 1,
  "weight_decay": 0.000001,
  "gradient_clip": 1.0
}
'''

with open('Configs/RNN3.json', 'w') as f:
    f.write(config_content)

print("Config file created: Configs/RNN3.json")

In [ ]:
# Verify data files exist
!ls -la Data/train/
!ls -la Data/val/
!ls -la Data/test/

In [ ]:
# Run a quick test to make sure the CoreAudioML modules can be imported
import sys
import os

# Ensure we're in the repository directory
current_dir = os.getcwd()
print(f"Starting from directory: {current_dir}")

# Try multiple possible paths
possible_paths = [
    'MonsterLSTM_Capture',  # If we're in /content
    'MonsterLSTM_Capture/MonsterLSTM_Capture',  # If nested
    'Automated-GuitarAmpModelling-3/MonsterLSTM_Capture',  # If old structure exists
    '.',  # If already in repo
]

repo_path = None
for path in possible_paths:
    if os.path.exists(os.path.join(path, 'CoreAudioML')):
        repo_path = os.path.abspath(path)
        print(f"✓ Found CoreAudioML in: {repo_path}")
        os.chdir(repo_path)
        break

if repo_path is None:
    print("⚠️  Searching for CoreAudioML...")
    # Walk up the directory tree
    check_dir = current_dir
    for _ in range(5):  # Check up to 5 levels up
        if os.path.exists(os.path.join(check_dir, 'CoreAudioML')):
            repo_path = check_dir
            print(f"✓ Found CoreAudioML in: {repo_path}")
            os.chdir(repo_path)
            break
        check_dir = os.path.dirname(check_dir)
    
    if repo_path is None:
        print("❌ Cannot find CoreAudioML!")
        print(f"Current directory: {os.getcwd()}")
        print(f"Contents: {os.listdir('.')[:10]}")

# Add current directory to Python path
sys.path.insert(0, os.getcwd())
print(f"✓ Python path includes: {os.getcwd()}")

try:
    import CoreAudioML.miscfuncs as miscfuncs
    import CoreAudioML.training as training
    import CoreAudioML.dataset as dataset
    import CoreAudioML.networks as networks
    print("✓ All CoreAudioML modules imported successfully!")
except Exception as e:
    print(f"✗ Import error: {e}")
    print(f"Current directory: {os.getcwd()}")
    print(f"CoreAudioML exists: {os.path.exists('CoreAudioML')}")
    if os.path.exists('CoreAudioML'):
        print(f"CoreAudioML contents: {os.listdir('CoreAudioML')}")
    import traceback
    traceback.print_exc()

In [ ]:
# Start training!
# This will run the main training script with the RNN3 configuration

# First, let's do a quick dry run to make sure everything works
print("Testing training setup with 1 epoch...")
!python dist_model_recnet.py --load_config RNN3 --epochs 1 --batch_size 64 --device dls2

# If the test works, you can run the full training:
# !python dist_model_recnet.py --load_config RNN3 --epochs 100 --device dls2

In [ ]:
# If you want to run the full training (this will take time), uncomment and run:
# !python dist_model_recnet.py --load_config RNN3 --epochs 200 --device dls2

# Or run with different parameters:
# !python dist_model_recnet.py --load_config RNN3 --epochs 100 --batch_size 1024 --hidden_size 64 --device dls2

In [ ]:
# Check training results
!ls -la Results/

# If training completed, you should see directories like 'dls2-RNN3' with model files
# You can download the results:
# from google.colab import files
# !zip -r results.zip Results/
# files.download('results.zip')

In [ ]:
# Monitor GPU usage during training (run this in a separate cell while training)
# !nvidia-smi --query-gpu=timestamp,name,utilization.gpu,utilization.memory,memory.used,memory.free --format=csv -l 5

## Tips for Better Training:

1. **GPU Memory**: If you get CUDA out of memory errors, reduce `batch_size` in the config
2. **Training Time**: Start with fewer epochs (50-100) to test, then increase
3. **Model Size**: Larger `hidden_size` (64, 128) gives better quality but slower training
4. **Data Quality**: Real guitar recordings give much better results than synthetic data
5. **Monitoring**: Use TensorBoard to monitor training progress

## Troubleshooting:

- **Import errors**: Make sure all files are uploaded/cloned correctly
- **CUDA errors**: Ensure GPU runtime is enabled
- **Memory errors**: Reduce batch size or use gradient accumulation
- **Data errors**: Check that WAV files are mono, 44.1kHz, and properly matched

## Next Steps:

1. Download trained models from the Results folder
2. Test inference with `proc_audio.py` on new audio
3. Experiment with different configurations (RNN1.json, RNN2.json)
4. Try different architectures or hyperparameters